In [1]:
#model 1
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Set seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Load cleaned data
df = pd.read_csv("/kaggle/input/questionnaire-data/clean_questionnaire_data.csv")
X = df.drop(columns=["target"]).values
y = df["target"].values

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Define neural network
class QuestionnaireNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(X_train.shape[1], 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Initialize model, loss, optimizer
model = QuestionnaireNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/50], Loss: {loss.item():.4f}")

# Evaluation on Test Set
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor).round()
    print("Test Set Evaluation:")
    print(classification_report(y_test_tensor.numpy(), preds.numpy()))

# Evaluation on Training Set
model.eval()
with torch.no_grad():
    train_preds = model(X_train_tensor).round()
    print("Training Set Evaluation:")
    print(classification_report(y_train_tensor.numpy(), train_preds.numpy()))


Epoch [10/50], Loss: 0.6219
Epoch [20/50], Loss: 0.5573
Epoch [30/50], Loss: 0.5053
Epoch [40/50], Loss: 0.4480
Epoch [50/50], Loss: 0.3964
Test Set Evaluation:
              precision    recall  f1-score   support

         0.0       0.70      0.66      0.68        35
         1.0       0.80      0.83      0.82        59

    accuracy                           0.77        94
   macro avg       0.75      0.74      0.75        94
weighted avg       0.76      0.77      0.76        94

Training Set Evaluation:
              precision    recall  f1-score   support

         0.0       0.80      0.78      0.79       158
         1.0       0.84      0.86      0.85       217

    accuracy                           0.83       375
   macro avg       0.82      0.82      0.82       375
weighted avg       0.83      0.83      0.83       375

